# Agent Guardrails and Human in the Loop (HITL)

Implement PII detection, content filtering, and human oversight for production agents.
## Why Guardrails Matter

Production agents need protection against:
- Leaking personally identifiable information (PII)
- Processing sensitive data (API keys, passwords)
- Inappropriate content generation
- Security vulnerabilities
- Compliance violations

**PII Middleware Strategies:**
1. **Redact**: Remove PII completely
2. **Mask**: Replace with placeholder (***)
3. **Block**: Prevent request from processing

## Strategy Reference

### PII Strategies
| Strategy | Effect | Use Case |
|----------|--------|----------|
| redact | Removes completely | PII that shouldn't be logged |
| mask | Replaces with *** | Preserve context while hiding data |
| block | Prevents processing | Critical secrets (API keys) |

### HITL Decisions
| Decision | Effect | Use Case |
|----------|--------|----------|
| approve | Execute as-is | Safe operations |
| edit | Modify then execute | Adjust parameters |
| reject | Block with feedback | Dangerous operations |

## Setup

In [1]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from scripts import base_tools

from langchain.agents.middleware import PIIMiddleware

from langgraph.checkpoint.memory import InMemorySaver

model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')
checkpointer = InMemorySaver()

system_prompt = """You are a helpful customer service assistant.
Assist users with their questions while protecting their privacy."""

## Setup checkpointer with SQLite
# from langgraph.checkpoint.sqlite import SqliteSaver
# import sqlite3
# conn = sqlite3.connect("db/middleware_agent.db", check_same_thread=False)
# checkpointer = SqliteSaver(conn)
# checkpointer.setup()

## PII Protection Strategies

One agent with different PII protection strategies.

In [11]:
# Agent with multiple PII protection strategies
agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=system_prompt,
    middleware=[
        # Strategy 1: BLOCK - Prevent processing if API keys detected
        # block will throw error i.e. block execution
        PIIMiddleware("api_key", detector=r"sk-[a-zA-Z0-9]{32}", strategy="block"), 
        
        # Strategy 2: REDACT - Remove emails completely
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        
        # Strategy 3: MASK - Replace credit cards with asterisks
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        
        # Additional protections
        PIIMiddleware("phone", detector=r"\d{3}-\d{3}-\d{4}", strategy="redact"),
        PIIMiddleware("url", strategy="redact", apply_to_input=True)
    ]
)


# Combined test with all PII types in a story format
response = agent.invoke({
    'messages': [HumanMessage("""
        Hello! I'm Laxmi Kant and I run KGP Talkie, an educational platform. 
        I'm reaching out because I need help setting up my account. 
        
        For contact purposes, you can reach me at udemy@kgptalkie.com or 
        call me at 555-123-4567. My website is https://kgptalkie.com where 
        I share AI and machine learning courses.
        
        I'd like to upgrade to a premium plan. My payment card number is 
        4532-1234-5678-9010. Also, I need to integrate the API into my 
        platform, so here's my API key: sk-1234567890abcdefghijklmnopqrstuv
        
        Looking forward to your assistance!
    """),
    HumanMessage("Tell me what information I had shared with you previously?")]
})

response

{'messages': [HumanMessage(content="\n        Hello! I'm Laxmi Kant and I run KGP Talkie, an educational platform. \n        I'm reaching out because I need help setting up my account. \n\n        For contact purposes, you can reach me at udemy@kgptalkie.com or \n        call me at 555-123-4567. My website is https://kgptalkie.com where \n        I share AI and machine learning courses.\n\n        I'd like to upgrade to a premium plan. My payment card number is \n        4532-1234-5678-9010. Also, I need to integrate the API into my \n        platform, so here's my API key: sk-1234567890abcdefghijklmnopqrstuv\n\n        Looking forward to your assistance!\n    ", additional_kwargs={}, response_metadata={}, id='fb1cdf65-7900-4cb2-94c0-88a81227a993'),
  HumanMessage(content='Tell me what information I had shared with you previously?', additional_kwargs={}, response_metadata={}, id='7bbd19f7-8d37-47f3-8679-08912839ca79'),
  AIMessage(content=[{'type': 'text', 'text': 'You previously share

In [5]:
response

{'messages': [HumanMessage(content="\n        Hello! I'm Laxmi Kant and I run KGP Talkie, an educational platform. \n        I'm reaching out because I need help setting up my account. \n\n        For contact purposes, you can reach me at udemy@kgptalkie.com or \n        call me at 555-123-4567. My website is https://kgptalkie.com where \n        I share AI and machine learning courses.\n\n        I'd like to upgrade to a premium plan. My payment card number is \n        4532-1234-5678-9010. Also, I need to integrate the API into my \n        platform, so here's my API key: sk-1234567890abcdefghijklmnopqrstuv\n\n        Looking forward to your assistance!\n    ", additional_kwargs={}, response_metadata={}, id='4196502c-733c-400c-89de-f41c34deed73'),
  HumanMessage(content='Tell me what information I had shared with you previously?', additional_kwargs={}, response_metadata={}, id='d7ec8bca-e7cc-409e-b4db-8d66a94929e3'),
  AIMessage(content=[{'type': 'text', 'text': 'You previously share

## Custom PII Patterns

Define domain-specific PII patterns.

In [8]:
# Agent with custom patterns
agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=system_prompt,
    middleware=[
        # Custom pattern: Employee IDs (EMP-123456)
        PIIMiddleware("employee_id", detector=r"EMP-\d{6}", strategy="mask"),
        
        # Custom pattern: Order IDs (ORD-ABC123)
        PIIMiddleware("order_id", detector=r"ORD-[A-Z0-9]{6}", strategy="redact"),
        
        # Standard patterns
        PIIMiddleware("email", strategy="redact"),
        PIIMiddleware("phone", detector=r"\d{3}-\d{3}-\d{4}", strategy="redact")
    ]
)

response = agent.invoke({
    'messages': [HumanMessage(
        "My employee ID is EMP-123456 and order ID is ORD-ABC123"
    )]
})

response

{'messages': [HumanMessage(content='My employee ID is ****3456 and order ID is [REDACTED_ORDER_ID]', additional_kwargs={}, response_metadata={}, id='c65bf1d1-62e4-498b-8be3-0c06cfde441b'),
  AIMessage(content=[{'type': 'text', 'text': "Thank you for providing your employee ID and order ID. I understand you're sharing this information for a reason, but please be assured that I cannot store or use this information. Your privacy is important. How can I assist you today?", 'extras': {'signature': 'CugCAXLI2nwEcSCkvS6hYHrmt0K64wnu0eU7z8rqi2G/IJq0iH7xZEMtNfw/HV0IPUwk3J+YUKppadyHGWIGntGJ7Ha4HNw1PFNvngWbTPDFavByAg7UctyOO4yGlQdZNVGqKE+1rKioewUoTLbKpezoZeagt98PnGhrpmf+5HrFtgm8NpyWMSSkPGpUy3PqSMrZLQbEFL0wTfp9MjaRucnejlOZlX4PrjQXLRUREaAgA8jxp5i3es1xVgBTf6IMKDuBFW2wgoACde7gA2AT4PgGCsX0nk/Eh01AsoKvOv9fFQrEgQoMxqaIiJV9LkV1ZB2EDDNafxaBF4GdEG2a9bjGqdzbn6e2UfTocQgOznCrKQKqWHnX03+UvfiBVmlNmPnQhEIstcvIi2ssiYHKvQPaaFR7iRrdgIm0nlgqAeO7Bj7MSmi13VO7HuTEG0d8nRPxE6aiek++pSXyKIYd2QbsxANQm1le6EJw'}}], additional_

## Human-in-the-Loop (HITL)

Add human approval for sensitive tool actions.

In [ ]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.tools import tool
from langgraph.types import Command

@tool
def write_file(path: str, content: str):
    """Write content to file."""
    try:
        with open(path, 'w') as f:
            f.write(content)
        return f"Successfully wrote to {path}"
    except Exception as e:
        return f"Error: {e}"

@tool
def execute_sql(query: str):
    """Execute SQL query."""
    return f"Would execute: {query}"

# Agent with HITL
agent = create_agent(
    model=model,
    tools=[write_file, execute_sql],
    checkpointer=checkpointer,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "write_file": True,  # All decisions: approve, edit, reject
                "execute_sql": {"allowed_decisions": ["approve", "reject"]},  # No editing
            },
            description_prefix="Tool execution pending approval",
        )
    ]
)

In [ ]:
# HITL Example 1: APPROVE action
config = {"configurable": {"thread_id": "hitl_approve"}}

result = agent.invoke({
    "messages": [HumanMessage("Write 'Hello World' to test.txt")]
}, config=config)

if "__interrupt__" in result:
    print("Interrupt:", result['__interrupt__'][0].value['action_requests'][0])
    
    # Approve
    result = agent.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config
    )
    print("\nApproved:", result['messages'][-1].content)

In [ ]:
# HITL Example 2: EDIT action
config = {"configurable": {"thread_id": "hitl_edit"}}

result = agent.invoke({
    "messages": [HumanMessage("Write 'Original' to data.txt")]
}, config=config)

if "__interrupt__" in result:
    print("Original:", result['__interrupt__'][0].value['action_requests'][0])
    
    # Edit before execution
    result = agent.invoke(
        Command(resume={
            "decisions": [{
                "type": "edit",
                "edited_action": {
                    "name": "write_file",
                    "args": {"path": "data.txt", "content": "Modified content"}
                }
            }]
        }),
        config=config
    )
    print("\nEdited:", result['messages'][-1].content)

In [ ]:
# HITL Example 3: REJECT action
config = {"configurable": {"thread_id": "hitl_reject"}}

result = agent.invoke({
    "messages": [HumanMessage("Delete all records from database")]
}, config=config)

if "__interrupt__" in result:
    print("Dangerous:", result['__interrupt__'][0].value['action_requests'][0])
    
    # Reject with feedback
    result = agent.invoke(
        Command(resume={
            "decisions": [{
                "type": "reject",
                "message": "Too dangerous. Use WHERE clause to specify records."
            }]
        }),
        config=config
    )
    print("\nRejected:", result['messages'][-1].content)